### *Podemos rodar às segundas-feiras já que as partidas da liga são jogadas aos domingos*

In [2]:
import pandas as pd
from functions import clube_id_e_sessao, pega_dados, conecta_ao_db

In [3]:
sessao, club_id= clube_id_e_sessao()

Clube ID: 79085


### Informações gerais dos jogos

In [4]:
# Conecta ao banco de dados
db= conecta_ao_db('dados_sokker.db')
curs= db.cursor()

#Pega informações únicas das partidas já adicionadas para que não sejam inseridos valores duplicados na base
matches = pd.read_sql_query("SELECT * FROM matches", db)

ids= matches['matchID'].unique()
data_esperada= matches['dateExpected'].unique()
data_iniciada= matches['dateStarted'].unique()

In [5]:
# Id do clube a serem pegas as informações
id_clube= club_id

# Pegando informações das partidas retirando aquelas que já foram pegas
url= f'https://sokker.org/xml/matches-team-{club_id}.xml'
df_matches= pega_dados(url, sessao, search= 'match')

#Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)
df_matches.dropna(inplace= True)

query= 'matchID not in @ids and dateExpected not in @data_esperada and dateStarted not in @data_iniciada'

# Retirando colunas que não salvaremos as informações
df_matches.drop(columns= ['homeTeamName', 'awayTeamName'], 
                inplace= True)

#df_matches= pd.concat([matches, df_matches])

df_matches= df_matches.query(query)
df_matches

,matchID,homeTeamID,awayTeamID,leagueID,round,season,week,day,dateExpected,dateStarted,homeTeamScore,awayTeamScore,supporters,weather,isFinished
34,42364291,31357,79085,1001,2,77,1132,6,2025-03-21 21:59,2025-03-21 21:59,0,2,4334,3,1
35,42617187,79085,51789,51530,0,0,1133,0,2025-03-22 19:35,2025-03-22 19:35,0,5,2306,2,1
36,42410239,80293,79085,1349,1,70,1133,1,2025-03-23 21:59,2025-03-23 21:59,0,2,9162,5,1
37,42617202,140828,79085,51530,0,0,1133,2,2025-03-24 22:00,2025-03-24 22:00,0,6,610,3,1
38,42617360,2927,79085,51530,0,0,1133,3,2025-03-25 22:00,2025-03-25 22:00,0,2,1632,7,1
39,42410251,79665,79085,1349,2,70,1133,4,2025-03-26 21:59,2025-03-26 21:59,0,7,6666,2,1


In [6]:
if len(df_matches) > 0:
    
    # Adiciona dados à tabela
    df_matches.to_sql('matches', con=db, if_exists='append', index= False)

In [7]:
#df_matches = pd.read_sql_query("SELECT * FROM matches", db)
len(df_matches)

6

### Estatísticas dos jogos

In [8]:
#Pega informações únicas das estatísticas já adicionadas para que não sejam inseridos valores duplicados na base
stats = pd.read_sql_query("SELECT * FROM matchStats", db)

ids= stats['matchID'].unique()
team= stats['teamID'].unique()
terco_defensivo= stats['timeOnHalf'].unique()
posse_de_bola= stats['timePossession'].unique()

In [9]:
#lista que guardará as estatísticas de cada partida
list_stats= list()

# Pegando estatísticas

for ID in df_matches['matchID'].unique():
    
    url= f'https://sokker.org/xml/match-{ID}.xml'
    df= pega_dados(url, sessao, search= 'teamStats')
    df['matchID']= ID
    list_stats.append(df)
    


df_stats= pd.concat([stats for stats in list_stats])

# Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)

df_stats.dropna(inplace= True)

query= 'matchID not in @ids and teamID not in @team and timeOnHalf not in @terco_defensivo and timePossession not in @posse_de_bola'

df_stats= df_stats.query(query)

df_stats

,teamID,timeOnHalf,timePossession,offsides,shoots,fouls,yellowCards,redCards,tacticName,ratingScoring,ratingPassing,ratingDefending,matchID
0,31357,40831,33604,0,4,2,1,0,442 Básica,5,5,4,42364291
1,79085,30288,37515,5,15,1,0,0,3-4-3BC,5,7,6,42364291
0,79085,47908,33771,2,1,3,1,0,5-4-1aSR,8,9,10,42617187
1,51789,22047,36184,1,12,0,0,0,AK new mir,11,10,9,42617187
0,80293,43211,31953,0,1,1,0,0,T04,5,8,8,42410239
1,79085,26741,37999,0,11,4,3,0,4-2-4a,10,9,9,42410239
0,140828,45998,34003,1,2,0,0,0,433,7,6,7,42617202
1,79085,24246,36241,0,23,1,1,0,3-4-3aQJ,10,9,11,42617202
0,2927,44946,28941,0,2,0,0,0,4-3-3 - wlasne,4,5,4,42617360
1,79085,25343,41348,0,26,5,2,0,4-4-2c,7,7,6,42617360


In [10]:
# Adicionando informações se houver alguma nova
if len(df_stats) > 0:
    
    # Adiciona dados à tabela
    df_stats.to_sql('matchStats', con=db, if_exists='append', index= False)

In [11]:
# Conferindo
stats = pd.read_sql_query("SELECT * FROM matchStats", db)
len(stats)

1644

### Estatísticas da Liga

In [12]:
#Pega informações ligas já adicionadas para que não sejam inseridos valores duplicados na base
league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)

ids= league_stats['leagueID'].unique()
team= league_stats['teamID'].unique()
rodada=  league_stats['round'].unique()

In [13]:
#Pegar o id da liga desejada entrando na liga na página do sokker
league_id= 1349

url= f'https://sokker.org/xml/league-{league_id}.xml'
df_league_stats= pega_dados(url, sessao, search= 'teams', tree_top= True)

In [14]:
df_league_stats['leagueID']= league_id
df_league_stats

,teamID,round,points,wins,draws,losses,goalsScored,goalsLost,rankTotal,leagueID
0,23755,2,6,2,0,0,13,0,6513013002010,1349
1,79085,2,6,2,0,0,9,0,6509009002004,1349
2,23775,2,6,2,0,0,9,0,6509009002001,1349
3,10520,2,6,2,0,0,8,1,6507008002011,1349
4,124139,2,4,1,1,0,2,1,4501002001006,1349
5,122169,2,4,1,1,0,2,1,4501002001005,1349
6,78671,2,3,1,0,1,3,2,3501003001008,1349
7,89569,2,0,0,0,2,1,4,497001000007,1349
8,79866,2,0,0,0,2,1,5,496001000009,1349
9,80293,2,0,0,0,2,0,5,495000000003,1349


In [15]:
# Adicionando informações 
if len(df_league_stats) > 0:
    
    # Adiciona dados à tabela
    df_league_stats.to_sql('leagueStats', con=db, if_exists='append', index= False)

league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)
len(league_stats)

### Liga informação geral

In [16]:
#Pega informações das ligas já adicionadas para que não sejam inseridos valores duplicados na base
leagues = pd.read_sql_query("SELECT * FROM league", db)

ids= leagues['leagueID'].unique()
league_name= leagues['name'].unique()

In [17]:
#Pegando dados da liga
df_league= pega_dados(url, sessao, search= 'info')

#Retirando coluna que não será utilizada
df_league.drop(columns= ['userID', 'round'], inplace= True)

#Retirando valores duplicados (se as estatísticas já tiverem sido adicionadas na base)
query= 'leagueID not in @ids and name not in @league_name'
df_league= df_league.query(query)

df_league

,leagueID,name,countryID,division,season,type,isOfficial,isCup


In [18]:
# Adicionando informações se for uma liga ainda não adicionada
if len(df_league) > 0:
    
    # Adiciona dados à tabela
    df_league.to_sql('league', con=db, if_exists='append', index= False)

In [19]:
league = pd.read_sql_query("SELECT * FROM league", db)
len(league)

4

In [20]:
# Salvando alterações
db.commit()

# Fechando conexão com a base de dados
db.close()